# Algorytmy tekstowe 2019/2020

# Laboratorium 5

# Autor - Łukasz Jezapkowicz

# Zadanie dotyczy wyszukiwania wzorców dwuwymiarowych.

# 0. Przygotowanie danych

Dla tak małego pliku jakim jest "haystack.txt" uznałem, że nie będzie mocno krzywdzące (pod względem wydajności) wypełnienie go wybranym symbolem '#' w celu osiągnięcia tekstu o równej długości wszystkich linii.

In [1]:
# otwieranie pliku
def open_file(file):
    file = open(file,mode='r', encoding="utf-8")
    data = file.read()
    file.close()
    return data

In [2]:
# wypełnianie pliku znakiem '#' (nie występuje ani razu) w celu równej długości każdej linii
def adapt_text(text):
    lines = text.splitlines()
    new_lines = []
    max_number_of_lines = 0
    for line in lines:
        if len(line) > max_number_of_lines:
            max_number_of_lines = len(line)
    
    for line in lines:
        new_lines.append(list(line.ljust(max_number_of_lines,'#')))
    
    return new_lines

Tekst przechowuje jako liste list w celu lepszego dostępu do jej elementów w algorytmie.

In [3]:
haystack_text = adapt_text(open_file("haystack.txt"))
print(str(haystack_text[0][:20]) + '...')
print(str(haystack_text[1][:20]) + '...')
print(str(haystack_text[2][:20]) + '...')
print('...')

['O', 'n', 'e', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 's', 'i', 'm', 'p', 'l', 'e', 's', 't', ' ']...
['o', 'f', ' ', 'w', 'r', 'i', 't', 't', 'e', 'n', ' ', 't', 'e', 'x', 't', 's', '.', ' ', 'T', 'h']...
['b', 'e', ' ', 'w', 'r', 'i', 't', 't', 'e', 'n', ' ', 'd', 'o', 'w', 'n', ' ', 'a', 's', ' ', 'a']...
...


Obraz "haystack.png" przechowywać będę jako macierz dwuwymiarową, w której każdy element zawiera albo 0 albo 1 (tryb czarno-biały)

In [5]:
# funkcja zwracająca zdjęcie jako tablicę dwuwymiarową o wartościach 0/1
def convert_to_binary_array(image):
    new_image = image.convert('P', palette=Image.ADAPTIVE, colors=256)
    A = array(new_image)
    data = []
    
    for i in range(len(A)):
        column = []
        for j in range(len(A[i])):
            column.append(A[i][j])
        data.append(column)
    return data

In [6]:
from PIL import Image
from numpy import empty,array

haystack_image = Image.open('haystack.png')
haystack_image_data = convert_to_binary_array(haystack_image)
            
print(str(haystack_image_data[0][:20]) + '...')
print(str(haystack_image_data[1][:20]) + '...')
print(str(haystack_image_data[2][:20]) + '...')
print('...')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]...
...


# 1. Zaimplementuj algorytm wyszukiwania wzorca 2-wymiarowego.

Na początku zaimplementuję funkcje, które budują automat dla podanej listy wzorców.

In [7]:
# klasa symbolizująca węzeł
class Node:
    def __init__(self, index):
        self.goto = {}
        self.out = []
        self.fail = None
        self.index = index

# funkcja tworząca nowy węzeł dla każdego znaku w każdym wzorcu
def create_forest(patterns):
    root = Node(0)
     
    i = 1
    for path in patterns:
        node = root
        for symbol in path:
            node = node.goto.setdefault(symbol, Node(i))
            i += 1
        node.out.append(path)
    return root

# funkcja budująca automat dla listy wzorców - Aho-Corasick
def build_automata(patterns):
    root = create_forest(patterns)
    queue = []
    for node in root.goto.values():
        queue.append(node)
        node.fail = root
 
    while len(queue) > 0:
        rnode = queue.pop(0)
 
        for key, unode in rnode.goto.items():
            queue.append(unode)
            fnode = rnode.fail
            while fnode != None and not key in fnode.goto:
                fnode = fnode.fail
            unode.fail = fnode.goto[key] if fnode else root
            unode.out += unode.fail.out
 
    return root

In [8]:
# Wyszukiwanie wzorca jednowymiarowego - z laboratorium 1
def KMPSearch(txt,pattern):
    S = set()
    K = []  
    t = -1
    K.append(t)
    for k in range(1, len(pattern) + 1):
        while(t >= 0 and pattern[t] != pattern[k - 1]):
            t = K[t]
        t = t + 1  
        K.append(t)
    m = 0  
    for i in range(0, len(txt)):
        while (m >= 0 and pattern[m] != txt[i]):
            m = K[m]
        m = m + 1  
        if m == len(pattern):
            S.add(i-m+1)
            m = K[m]
    return len(S), S

In [9]:
from time import time
# funkcja wyszukująca wzorców dwuwymiarowych, patterns to lista kolejnych kolumn wzorca 2d
def search_patterns(text,patterns):
    time1 = time()
    automata = build_automata(patterns)
    
    # numery kolejnych stanów prawidłowych
    patterns_pattern = []
    p_list = []
    current = 0
    for pattern in patterns:
        current += len(pattern)
        num = current
        for p in p_list:
            if p[0] == pattern:
                num = p[1]
                
        patterns_pattern.append(num)
        if num == current:
            p_list.append((pattern,num))
        
    states = [[0 for _ in range(len(text[0]))] for _ in range(len(text))]
        
    # wypełniam macierz stanów dla kolejnych kolumn
    for j in range(len(text[0])):
        # kolumna j
        node = automata
        for i in range(len(text)):
            while node != None and not text[i][j] in node.goto:
                node = node.fail
            if node == None:
                node = automata
                continue
            node = node.goto[text[i][j]]
            states[i][j] = node.index
    
    time2 = time()
    # dla każdego wiersza macierzy stanów poszukuje "patterns_pattern"
    i = -1 * len(patterns[0]) + 1
    count = 0
    for row in states:
        lenS, S = KMPSearch(row,patterns_pattern)
        count += lenS
        for j in S:
            print("Pattern found: (" + str(i) + "," + str(j) + ") ... (" + str(i) + "," + str(j+len(patterns)-1) + ")")
            print("               ...")
            print("               (" + str(i+len(patterns[0])-1) + "," + str(j) + ") ... (" + \
                  str(i+len(patterns[0])-1) + "," + str(j+len(patterns)-1) + ")")
        i += 1
    print("Total number of pattern occurences = " + str(count))
    time3 = time()
    
    print("Automata building time => " + str(1000 * (time2-time1)) + " milliseconds")
    print("Searching time => " + str(1000 * (time3-time2)) + " milliseconds")
    return count

# 2. Znajdź w załączonym pliku "haystack.txt" wszyskie sytuacje, gdy taka sama litera występuje na tej samej pozycji w dwóch kolejnych linijkach. Zwróć uwagę, na nierówną długość linii w pliku.

Z problemem nierównej długości linii w pliku uporałem się już w podpunkcie 0. Nie będę więc sobie nim już więcej zawracał głowy.  
  
  
Żeby nie wykonywać algorytmu 256 razy, na początku w pojedynczym przejściu znajdę wszystkie litery, które występują w tym tekście.

In [10]:
letters = set()
for row in haystack_text:
    for letter in row:
        letters.add(letter)

Teraz dla każdej występującej literki $x$ wyszukam wzorca $[[x],[x]]$ czyli pojedynczej kolumny o dwóch wierszach i wartościach $x$.  
  
Ponieważ moja funkcja przyjmuje liste wzorców, które są kolejnymi kolumnami wzorca dwuwymiarowego to do funkcji przekażę liste o jednym elemencie $[xx]$

In [12]:
count = 0
for letter in letters:
    print("\nLetter '" + letter + "':")
    count += search_patterns(haystack_text,[letter+letter])


Letter 'x':
Pattern found: (28,68) ... (28,68)
               ...
               (29,68) ... (29,68)
Total number of pattern occurences = 1
Automata building time => 1.995086669921875 milliseconds
Searching time => 2.023935317993164 milliseconds

Letter 'w':
Pattern found: (1,3) ... (1,3)
               ...
               (2,3) ... (2,3)
Pattern found: (21,70) ... (21,70)
               ...
               (22,70) ... (22,70)
Total number of pattern occurences = 2
Automata building time => 1.024007797241211 milliseconds
Searching time => 2.991914749145508 milliseconds

Letter 'a':
Pattern found: (0,82) ... (0,82)
               ...
               (1,82) ... (1,82)
Pattern found: (3,30) ... (3,30)
               ...
               (4,30) ... (4,30)
Pattern found: (5,60) ... (5,60)
               ...
               (6,60) ... (6,60)
Pattern found: (6,63) ... (6,63)
               ...
               (7,63) ... (7,63)
Pattern found: (20,6) ... (20,6)
               ...
               (21,6

               ...
               (32,104) ... (32,104)
Pattern found: (32,96) ... (32,96)
               ...
               (33,96) ... (33,96)
Pattern found: (32,97) ... (32,97)
               ...
               (33,97) ... (33,97)
Pattern found: (32,98) ... (32,98)
               ...
               (33,98) ... (33,98)
Pattern found: (32,99) ... (32,99)
               ...
               (33,99) ... (33,99)
Pattern found: (32,100) ... (32,100)
               ...
               (33,100) ... (33,100)
Pattern found: (32,101) ... (32,101)
               ...
               (33,101) ... (33,101)
Pattern found: (32,102) ... (32,102)
               ...
               (33,102) ... (33,102)
Pattern found: (32,103) ... (32,103)
               ...
               (33,103) ... (33,103)
Pattern found: (32,104) ... (32,104)
               ...
               (33,104) ... (33,104)
Pattern found: (32,88) ... (32,88)
               ...
               (33,88) ... (33,88)
Pattern found: (32,89) ... (32,89)

               ...
               (71,98) ... (71,98)
Pattern found: (70,99) ... (70,99)
               ...
               (71,99) ... (71,99)
Pattern found: (70,100) ... (70,100)
               ...
               (71,100) ... (71,100)
Pattern found: (70,101) ... (70,101)
               ...
               (71,101) ... (71,101)
Pattern found: (70,102) ... (70,102)
               ...
               (71,102) ... (71,102)
Pattern found: (70,103) ... (70,103)
               ...
               (71,103) ... (71,103)
Pattern found: (70,104) ... (70,104)
               ...
               (71,104) ... (71,104)
Pattern found: (71,76) ... (71,76)
               ...
               (72,76) ... (72,76)
Pattern found: (71,77) ... (71,77)
               ...
               (72,77) ... (72,77)
Pattern found: (71,78) ... (71,78)
               ...
               (72,78) ... (72,78)
Pattern found: (71,79) ... (71,79)
               ...
               (72,79) ... (72,79)
Pattern found: (71,80) ... (71,80)
 

               (32,1) ... (32,1)
Pattern found: (35,18) ... (35,18)
               ...
               (36,18) ... (36,18)
Pattern found: (51,32) ... (51,32)
               ...
               (52,32) ... (52,32)
Pattern found: (54,33) ... (54,33)
               ...
               (55,33) ... (55,33)
Pattern found: (56,13) ... (56,13)
               ...
               (57,13) ... (57,13)
Pattern found: (64,29) ... (64,29)
               ...
               (65,29) ... (65,29)
Pattern found: (67,40) ... (67,40)
               ...
               (68,40) ... (68,40)
Pattern found: (67,57) ... (67,57)
               ...
               (68,57) ... (68,57)
Pattern found: (67,35) ... (67,35)
               ...
               (68,35) ... (68,35)
Total number of pattern occurences = 15
Automata building time => 1.031637191772461 milliseconds
Searching time => 3.956317901611328 milliseconds

Letter '"':
Total number of pattern occurences = 0
Automata building time => 2.0208358764648438 milliseconds

In [13]:
print("In total, there were " + str(count) + " occurences of such situations in text")

In total, there were 2420 occurences of such situations in text


Jak widać, jest to dosyć częsta sytuacja.  
  
Po ręcznym sprawdzeniu kilku wartości stwierdzam, iż algorytm odszukuje poprawnie taki wzorzec.

# 3. Znajdź wszystkie wystąpienia "th" oraz "t h" w dwóch kolejnych liniach na tej samej pozycji.

Zacznijmy od "th" w dwóch kolejnych liniach na tej samej pozycji.  
  
Jako liste wzorców podaje kolejne kolumny wzorca czyli $[tt,hh]$

In [14]:
count = search_patterns(haystack_text,['tt','hh'])

Total number of pattern occurences = 0
Automata building time => 1.9681453704833984 milliseconds
Searching time => 2.0208358764648438 milliseconds


Ręczne przeszukanie pliku potwierdza, iż faktycznie nie ma takiej sytuacji w tym pliku.  
  
Teraz dla "t h" w dwóch kolejnych liniach na tej samej pozycji.  
  
Jako liste wzorców podaje kolejne kolumny wzorca czyli $[tt,--,hh]$, gdzie $-$ symbolizuje pusty znak.

In [15]:
count = search_patterns(haystack_text,['tt','  ','hh'])

Pattern found: (37,0) ... (37,2)
               ...
               (38,0) ... (38,2)
Total number of pattern occurences = 1
Automata building time => 1.966714859008789 milliseconds
Searching time => 2.9921531677246094 milliseconds


Ręczne przeszukanie pliku potwierdza, iż faktycznie taka sytuacja występuje tylko raz, w miejscu wskazanym przez algorytm.  
  
Stwierdzam zatem, iż algorytm działa poprawnie dla pliku tekstowego "haystack.txt".

# 4. Wybierz przynajmniej 4 litery (małe). Znajdź wszystkie wystąpienia tej litery w załączonym pliku "haystack.png".

Na początku manipulując metodą Image.crop postaram się wyciąć odpowiednie wzorce dla małych liter.  
  
Literki ścinałem tak długo jak długo program zwracał niepoprawny wynik.

In [16]:
from IPython.display import Image

In [17]:
# funkcja zwracajaca odpowiedni wycinek macierzy
def get_data(left,upper,right,lower):
    data = []
    for row in haystack_image_data[upper:lower]:
        data.append(row[left:right])
    return data

Litera e:

In [18]:
pattern1 = haystack_image.crop((591,565,600,575))
pattern1_data = get_data(591,565,600,575)
Image(filename='e.png')

Litera c:

In [19]:
pattern2 = haystack_image.crop((486,564,494,575))
pattern2_data = get_data(486,564,494,575)
Image(filename='c.png')

Litera o:

In [20]:
pattern3 = haystack_image.crop((214,564,223,576))
pattern3_data = get_data(214,564,223,576)
Image(filename='o.png')

Litera u:

In [21]:
pattern4 = haystack_image.crop((608,124,617,137))
pattern4_data = get_data(608,124,617,137)
Image(filename='u.png')

Litera x:

In [22]:
pattern5 = haystack_image.crop((366,231,376,247))
pattern5_data = get_data(366,231,376,247)
Image(filename='x.png')

Litera a:

In [23]:
pattern6 = haystack_image.crop((120,234,129,245))
pattern6_data = get_data(120,234,129,245)
Image(filename='a.png')

In [24]:
# funkcja zwracająca kolumny podanego wzorca
def get_patterns_columns(data):
    patterns = []
    for j in range(len(data[0])):
        column = []
        for i in range(len(data)):
            column.append(data[i][j])
        patterns.append(column)
    return patterns

In [25]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern1_data))

Pattern found: (37,546) ... (37,554)
               ...
               (46,546) ... (46,554)
Pattern found: (37,741) ... (37,749)
               ...
               (46,741) ... (46,749)
Pattern found: (37,110) ... (37,118)
               ...
               (46,110) ... (46,118)
Pattern found: (37,175) ... (37,183)
               ...
               (46,175) ... (46,183)
Pattern found: (37,595) ... (37,603)
               ...
               (46,595) ... (46,603)
Pattern found: (37,52) ... (37,60)
               ...
               (46,52) ... (46,60)
Pattern found: (37,381) ... (37,389)
               ...
               (46,381) ... (46,389)
Pattern found: (37,575) ... (37,583)
               ...
               (46,575) ... (46,583)
Pattern found: (59,247) ... (59,255)
               ...
               (68,247) ... (68,255)
Pattern found: (59,556) ... (59,564)
               ...
               (68,556) ... (68,564)
Pattern found: (59,438) ... (59,446)
               ...
               (68

               ...
               (860,159) ... (860,167)
Pattern found: (873,321) ... (873,329)
               ...
               (882,321) ... (882,329)
Pattern found: (873,449) ... (873,457)
               ...
               (882,449) ... (882,457)
Pattern found: (873,706) ... (873,714)
               ...
               (882,706) ... (882,714)
Pattern found: (873,292) ... (873,300)
               ...
               (882,292) ... (882,300)
Pattern found: (873,268) ... (873,276)
               ...
               (882,268) ... (882,276)
Pattern found: (873,341) ... (873,349)
               ...
               (882,341) ... (882,349)
Pattern found: (873,662) ... (873,670)
               ...
               (882,662) ... (882,670)
Pattern found: (873,607) ... (873,615)
               ...
               (882,607) ... (882,615)
Pattern found: (895,72) ... (895,80)
               ...
               (904,72) ... (904,80)
Pattern found: (895,248) ... (895,256)
               ...
               

Pattern found: (1577,163) ... (1577,171)
               ...
               (1586,163) ... (1586,171)
Pattern found: (1577,549) ... (1577,557)
               ...
               (1586,549) ... (1586,557)
Pattern found: (1577,231) ... (1577,239)
               ...
               (1586,231) ... (1586,239)
Pattern found: (1577,39) ... (1577,47)
               ...
               (1586,39) ... (1586,47)
Pattern found: (1577,527) ... (1577,535)
               ...
               (1586,527) ... (1586,535)
Pattern found: (1577,278) ... (1577,286)
               ...
               (1586,278) ... (1586,286)
Pattern found: (1577,662) ... (1577,670)
               ...
               (1586,662) ... (1586,670)
Pattern found: (1577,473) ... (1577,481)
               ...
               (1586,473) ... (1586,481)
Pattern found: (1577,731) ... (1577,739)
               ...
               (1586,731) ... (1586,739)
Pattern found: (1599,66) ... (1599,74)
               ...
               (1608,66) ... (1608,74

In [26]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern2_data))

Pattern found: (58,690) ... (58,697)
               ...
               (68,690) ... (68,697)
Pattern found: (58,370) ... (58,377)
               ...
               (68,370) ... (68,377)
Pattern found: (58,420) ... (58,427)
               ...
               (68,420) ... (68,427)
Pattern found: (58,589) ... (58,596)
               ...
               (68,589) ... (68,596)
Pattern found: (80,328) ... (80,335)
               ...
               (90,328) ... (90,335)
Pattern found: (80,428) ... (80,435)
               ...
               (90,428) ... (90,435)
Pattern found: (80,506) ... (80,513)
               ...
               (90,506) ... (90,513)
Pattern found: (80,378) ... (80,385)
               ...
               (90,378) ... (90,385)
Pattern found: (80,670) ... (80,677)
               ...
               (90,670) ... (90,677)
Pattern found: (102,681) ... (102,688)
               ...
               (112,681) ... (112,688)
Pattern found: (102,307) ... (102,314)
               ...
        

Pattern found: (960,685) ... (960,692)
               ...
               (970,685) ... (970,692)
Pattern found: (960,526) ... (960,533)
               ...
               (970,526) ... (970,533)
Pattern found: (960,494) ... (960,501)
               ...
               (970,494) ... (970,501)
Pattern found: (960,663) ... (960,670)
               ...
               (970,663) ... (970,670)
Pattern found: (960,26) ... (960,33)
               ...
               (970,26) ... (970,33)
Pattern found: (982,369) ... (982,376)
               ...
               (992,369) ... (992,376)
Pattern found: (982,530) ... (982,537)
               ...
               (992,530) ... (992,537)
Pattern found: (1004,689) ... (1004,696)
               ...
               (1014,689) ... (1014,696)
Pattern found: (1004,679) ... (1004,686)
               ...
               (1014,679) ... (1014,686)
Pattern found: (1026,289) ... (1026,296)
               ...
               (1036,289) ... (1036,296)
Pattern found: (1026,3

In [27]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern3_data))

Pattern found: (36,451) ... (36,459)
               ...
               (47,451) ... (47,459)
Pattern found: (36,68) ... (36,76)
               ...
               (47,68) ... (47,76)
Pattern found: (36,646) ... (36,654)
               ...
               (47,646) ... (47,654)
Pattern found: (36,406) ... (36,414)
               ...
               (47,406) ... (47,414)
Pattern found: (36,510) ... (36,518)
               ...
               (47,510) ... (47,518)
Pattern found: (58,26) ... (58,34)
               ...
               (69,26) ... (69,34)
Pattern found: (58,263) ... (58,271)
               ...
               (69,263) ... (69,271)
Pattern found: (80,225) ... (80,233)
               ...
               (91,225) ... (91,233)
Pattern found: (80,355) ... (80,363)
               ...
               (91,355) ... (91,363)
Pattern found: (80,134) ... (80,142)
               ...
               (91,134) ... (91,142)
Pattern found: (102,473) ... (102,481)
               ...
               (113,

Pattern found: (806,193) ... (806,201)
               ...
               (817,193) ... (817,201)
Pattern found: (806,324) ... (806,332)
               ...
               (817,324) ... (817,332)
Pattern found: (806,134) ... (806,142)
               ...
               (817,134) ... (817,142)
Pattern found: (806,657) ... (806,665)
               ...
               (817,657) ... (817,665)
Pattern found: (806,532) ... (806,540)
               ...
               (817,532) ... (817,540)
Pattern found: (806,607) ... (806,615)
               ...
               (817,607) ... (817,615)
Pattern found: (828,96) ... (828,104)
               ...
               (839,96) ... (839,104)
Pattern found: (828,34) ... (828,42)
               ...
               (839,34) ... (839,42)
Pattern found: (828,551) ... (828,559)
               ...
               (839,551) ... (839,559)
Pattern found: (828,363) ... (828,371)
               ...
               (839,363) ... (839,371)
Pattern found: (828,252) ... (828,26

Total number of pattern occurences = 369
Automata building time => 852.759599685669 milliseconds
Searching time => 439.78238105773926 milliseconds


In [28]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern4_data))

Pattern found: (36,293) ... (36,301)
               ...
               (48,293) ... (48,301)
Pattern found: (80,495) ... (80,503)
               ...
               (92,495) ... (92,503)
Pattern found: (80,637) ... (80,645)
               ...
               (92,637) ... (92,645)
Pattern found: (80,295) ... (80,303)
               ...
               (92,295) ... (92,303)
Pattern found: (124,312) ... (124,320)
               ...
               (136,312) ... (136,320)
Pattern found: (124,588) ... (124,596)
               ...
               (136,588) ... (136,596)
Pattern found: (124,608) ... (124,616)
               ...
               (136,608) ... (136,616)
Pattern found: (124,470) ... (124,478)
               ...
               (136,470) ... (136,478)
Pattern found: (146,202) ... (146,210)
               ...
               (158,202) ... (158,210)
Pattern found: (168,454) ... (168,462)
               ...
               (180,454) ... (180,462)
Pattern found: (190,610) ... (190,618)
       

In [29]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern5_data))

Pattern found: (55,135) ... (55,144)
               ...
               (70,135) ... (70,144)
Pattern found: (99,139) ... (99,148)
               ...
               (114,139) ... (114,148)
Pattern found: (99,238) ... (99,247)
               ...
               (114,238) ... (114,247)
Pattern found: (121,419) ... (121,428)
               ...
               (136,419) ... (136,428)
Pattern found: (143,342) ... (143,351)
               ...
               (158,342) ... (158,351)
Pattern found: (165,297) ... (165,306)
               ...
               (180,297) ... (180,306)
Pattern found: (187,314) ... (187,323)
               ...
               (202,314) ... (202,323)
Pattern found: (209,410) ... (209,419)
               ...
               (224,410) ... (224,419)
Pattern found: (209,254) ... (209,263)
               ...
               (224,254) ... (224,263)
Pattern found: (231,366) ... (231,375)
               ...
               (246,366) ... (246,375)
Pattern found: (231,447) ... (231,456)

In [30]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern6_data))

Pattern found: (36,323) ... (36,331)
               ...
               (46,323) ... (46,331)
Pattern found: (36,487) ... (36,495)
               ...
               (46,487) ... (46,495)
Pattern found: (36,263) ... (36,271)
               ...
               (46,263) ... (46,271)
Pattern found: (36,623) ... (36,631)
               ...
               (46,623) ... (46,631)
Pattern found: (36,207) ... (36,215)
               ...
               (46,207) ... (46,215)
Pattern found: (36,751) ... (36,759)
               ...
               (46,751) ... (46,759)
Pattern found: (58,578) ... (58,586)
               ...
               (68,578) ... (68,586)
Pattern found: (58,642) ... (58,650)
               ...
               (68,642) ... (68,650)
Pattern found: (58,391) ... (58,399)
               ...
               (68,391) ... (68,399)
Pattern found: (58,333) ... (58,341)
               ...
               (68,333) ... (68,341)
Pattern found: (58,303) ... (58,311)
               ...
              

               ...
               (838,601) ... (838,609)
Pattern found: (850,56) ... (850,64)
               ...
               (860,56) ... (860,64)
Pattern found: (850,243) ... (850,251)
               ...
               (860,243) ... (860,251)
Pattern found: (872,56) ... (872,64)
               ...
               (882,56) ... (882,64)
Pattern found: (872,369) ... (872,377)
               ...
               (882,369) ... (882,377)
Pattern found: (872,641) ... (872,649)
               ...
               (882,641) ... (882,649)
Pattern found: (894,106) ... (894,114)
               ...
               (904,106) ... (904,114)
Pattern found: (894,226) ... (894,234)
               ...
               (904,226) ... (904,234)
Pattern found: (916,347) ... (916,355)
               ...
               (926,347) ... (926,355)
Pattern found: (916,229) ... (916,237)
               ...
               (926,229) ... (926,237)
Pattern found: (938,504) ... (938,512)
               ...
               (948

Pattern found: (1730,226) ... (1730,234)
               ...
               (1740,226) ... (1740,234)
Pattern found: (1730,387) ... (1730,395)
               ...
               (1740,387) ... (1740,395)
Pattern found: (1730,617) ... (1730,625)
               ...
               (1740,617) ... (1740,625)
Pattern found: (1730,508) ... (1730,516)
               ...
               (1740,508) ... (1740,516)
Pattern found: (1730,302) ... (1730,310)
               ...
               (1740,302) ... (1740,310)
Pattern found: (1730,559) ... (1730,567)
               ...
               (1740,559) ... (1740,567)
Pattern found: (1730,685) ... (1730,693)
               ...
               (1740,685) ... (1740,693)
Pattern found: (1730,438) ... (1730,446)
               ...
               (1740,438) ... (1740,446)
Pattern found: (1730,343) ... (1730,351)
               ...
               (1740,343) ... (1740,351)
Pattern found: (1730,284) ... (1730,292)
               ...
               (1740,284) ... (

Podsumowanie:  
  
Litera e:  Ilość wystąpień w tekście (Notepad) = 583, Wartość zwrócona przez program = 583  
  
Litera c:  Ilość wystąpień w tekście (Notepad) = 213, Wartość zwrócona przez program = 213  
  
Litera o:  Ilość wystąpień w tekście (Notepad) = 369, Wartość zwrócona przez program = 369  
  
Litera u:  Ilość wystąpień w tekście (Notepad) = 106, Wartość zwrócona przez program = 106  
  
Litera x:  Ilość wystąpień w tekście (Notepad) = 57, Wartość zwrócona przez program = 57  
  
Litera a:  Ilość wystąpień w tekście (Notepad) = 397, Wartość zwrócona przez program = 397  
  
Wszystko zatem działa zgodnie z założeniami.

# 5. Znajdź wszystkie wystąpienia słowa "p a t t e r n" w haystack.png.

In [31]:
pattern7 = haystack_image.crop((422,494,530,515))
pattern7_data = get_data(422,494,530,515)
Image(filename='pattern.png')

In [32]:
count = search_patterns(haystack_image_data,get_patterns_columns(pattern7_data))

Pattern found: (472,178) ... (472,285)
               ...
               (492,178) ... (492,285)
Pattern found: (494,422) ... (494,529)
               ...
               (514,422) ... (514,529)
Pattern found: (538,236) ... (538,343)
               ...
               (558,236) ... (558,343)
Pattern found: (582,136) ... (582,243)
               ...
               (602,136) ... (602,243)
Pattern found: (626,242) ... (626,349)
               ...
               (646,242) ... (646,349)
Total number of pattern occurences = 5
Automata building time => 904.5541286468506 milliseconds
Searching time => 466.78972244262695 milliseconds


Również tutaj Notepad zwraca wartość identyczną z programem (5) co pokazuje jego poprawność.

# 6. Porównaj czas budowania automatu i czas wyszukiwania dla różnych rozmiarów wzorca.

In [33]:
print(len(haystack_text))
pat1 = ['abc','bcd','dcd','123']
count = search_patterns(haystack_text,pat1)

84
Total number of pattern occurences = 0
Automata building time => 2.0203590393066406 milliseconds
Searching time => 1.9943714141845703 milliseconds


Jak widać czas budowania automatu jest mały dla takiego małego wzorca. Czas wyszukiwania również jest bardzo krótki.

In [34]:
pat2 = []
tmp = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
for i in range(100):
    pat2.append(tmp)
count = search_patterns(haystack_text,pat2)

Total number of pattern occurences = 0
Automata building time => 3.988504409790039 milliseconds
Searching time => 4.01759147644043 milliseconds


Czas budowania automatu dla większego wzorca jest nieco większy lecz wciąż mały. Czas wyszukiwania zwiększa się, ponieważ  
wyszukiwany wzorzec jednowymiarowy jest dłuższy.

In [36]:
pat3 = []
for j in range(len(haystack_text[0])):
    column = ''
    for i in range(len(haystack_text)):
        column += haystack_text[i][j]
    pat3.append(column)
count = search_patterns(haystack_text,pat3)

Pattern found: (0,0) ... (0,104)
               ...
               (83,0) ... (83,104)
Total number of pattern occurences = 1
Automata building time => 19.974231719970703 milliseconds
Searching time => 4.959821701049805 milliseconds


Czas budowania automatu dla wzorca, który jest całym tekstem jest zauważalnie dłuższy. Czas wyszukiwania nieco zwiększył się z tego samego powodu co powyżej.

In [37]:
pat4 = []
tmp = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
for i in range(1000):
    pat4.append(tmp)
count = search_patterns(haystack_text,pat4)

Total number of pattern occurences = 0
Automata building time => 24.933576583862305 milliseconds
Searching time => 23.935556411743164 milliseconds


Czas budowania automatu jest podobny co powyżej, lecz czas wyszukiwania jest dłuższy z powodu większej ilości kolumn.

In [38]:
pat5 = []
tmp = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
for i in range(100000):
    pat5.append(tmp)
count = search_patterns(haystack_text,pat5)

Total number of pattern occurences = 0
Automata building time => 1919.9018478393555 milliseconds
Searching time => 1846.069574356079 milliseconds


Czas budowania automatu i wyszukiwania jest dłuższy niemal 100 razy (100 razy więcej kolumn) -> pokazuje to liniowość rozwiązania.

# 7. Podziel plik na 2, 4 i 8 fragmentów (w poziomie) i porównaj czas przeszukiwania.

W celu wykonania tego doświadczenia zmodyfikuję lekko funkcję search_patterns żeby "symulowała" wielowątkowość.

In [39]:
from math import ceil
# funkcja wyszukująca wzorców dwuwymiarowych, patterns to lista kolejnych kolumn wzorca 2d
def search_patterns_threads(text,patterns,threads):
    time1 = time()
    automata = build_automata(patterns)
    
    # numery kolejnych stanów prawidłowych
    patterns_pattern = []
    p_list = []
    current = 0
    for pattern in patterns:
        current += len(pattern)
        num = current
        for p in p_list:
            if p[0] == pattern:
                num = p[1]
                
        patterns_pattern.append(num)
        if num == current:
            p_list.append((pattern,num))
        
    states = [[0 for _ in range(len(text[0]))] for _ in range(len(text))]
        
    # wypełniam macierz stanów dla kolejnych kolumn
    for j in range(len(text[0])):
        # kolumna j
        node = automata
        for i in range(len(text)):
            while node != None and not text[i][j] in node.goto:
                node = node.fail
            if node == None:
                node = automata
                continue
            node = node.goto[text[i][j]]
            states[i][j] = node.index
    
    time2 = time()
    # dla każdego wiersza macierzy stanów poszukuje "patterns_pattern"
    i = -1 * len(patterns[0]) + 1
    count = 0
    max_time = 0
    for t in range(threads):
        time3 = time()
        for row in states[t*ceil(len(states)/threads):(t+1)*ceil(len(states)/threads)]:
            lenS, S = KMPSearch(row,patterns_pattern)
            count += lenS
            for j in S:
                print("Pattern found: (" + str(i) + "," + str(j) + ") ... (" + str(i) + "," + str(j+len(patterns)-1) + ")")
                print("               ...")
                print("               (" + str(i+len(patterns[0])-1) + "," + str(j) + ") ... (" + \
                      str(i+len(patterns[0])-1) + "," + str(j+len(patterns)-1) + ")")
            i += 1
        time4 = time()
        if (time4-time3 > max_time):
            max_time = time4-time3
            
    print("Total number of pattern occurences = " + str(count))    
    print("Automata building time => " + str(1000 * (time2-time1)) + " milliseconds")
    print("Searching time (longest thread time) => " + str(1000 * max_time) + " milliseconds")
    return count

In [40]:
count = search_patterns_threads(haystack_text,pat5,2)

Total number of pattern occurences = 0
Automata building time => 1958.8768482208252 milliseconds
Searching time (longest thread time) => 919.5289611816406 milliseconds


In [41]:
count = search_patterns_threads(haystack_text,pat5,4)

Total number of pattern occurences = 0
Automata building time => 1928.8666248321533 milliseconds
Searching time (longest thread time) => 485.7141971588135 milliseconds


In [42]:
count = search_patterns_threads(haystack_text,pat5,8)

Total number of pattern occurences = 0
Automata building time => 1903.9061069488525 milliseconds
Searching time (longest thread time) => 242.356538772583 milliseconds


Jak widać wszystko działa zgodnie z założeniem -> czas wyszukiwania zmniejsza się około 2 razy wraz z dwukrotnym zwiększeniem ilości wątków.

# Wniosek

Zadanie pozwoliło zapoznać się z mechanizmem wyszukiwania wzorców dwuwymiarowym, problemem, który jest mocno rozchwytywany i wykorzystywany w wielu dziedzinach. Zastosowany algorytm można wykorzystać zarówno do plików tekstowych jak i graficznych (każdego pliku, który możemy w jakiś sposób zrzutować np. na macierz). Ostatnie polecenie pokazało, że łatwo zrównoleglić obliczenia (tutaj wyszukiwanie wzorca jednowymiarowego). Można również zrównoleglić wypełnianie macierzy stanów, ponieważ wyniki dla każdej kolumny są od siebie niezależne.